In [ ]:
%%capture
!rm -r ./*
!git clone https://github.com/Mamiglia/BNN_Human_motion
!mv BNN_Human_motion/* .
!pip install lightning bayesian-torch

# Training Notebook
In this notebook call the training functions on all the architectures, then save the models


In [ ]:
input_n  =10 # number of frames to train on (default=10)
output_n =25 # number of frames to predict on
input_dim=3  # dimensions of the input coordinates(default=3)
skip_rate=1  # skip rate of frames
joints_to_consider=22

In [ ]:
from funcs import load_dataset

trainset, valset = load_dataset(input_n, output_n, skip_rate, 256)

## Transformer

In [ ]:
from funcs import STTFormerBayes, LitSTTFormerBayes

model = LitSTTFormerBayes(num_joints=joints_to_consider,
                 num_frames=input_n, num_frames_out=output_n, num_heads=5,
                 num_channels=3, kernel_size=[5,5], use_pes=True)

In [ ]:
import lightning as L
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

early_stop_callback = EarlyStopping(monitor="validation_loss", min_delta=0.5, patience=5, verbose=True)
trainer = L.Trainer(
    accelerator='gpu',
    default_root_dir="logs/transformer/",
    callbacks = [early_stop_callback],
    gradient_clip_val = 0.5,
    max_epochs=35)

trainer.fit(model=model, train_dataloaders=trainset, val_dataloaders=valset)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/transformer/

## Save model parameters